In [ ]:
import cv2
import glob
import os
import pandas as pd
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from skimage import io


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.callbacks import *

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import activations

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
sess = tf.compat.v1.Session
print(sess)

In [ ]:
with tf.device('/gpu:0'):
 a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
 b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
 c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

In [ ]:
tf.test.gpu_device_name()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/GPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

In [ ]:
print(keras.__version__)

In [ ]:
print(tf.__version__)

In [ ]:
#Defining Path
data_path = '../input/diabeticretino-255/DR_Classify_256'
classes = ['0', '1','2', '3','4']
num_classes = len(classes)

In [ ]:
img_size = (256,256)
channels = 3
batch_size =16
input_size = (img_size[0], img_size[1], channels)

In [ ]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    
    def __init__(self, mode='train', ablation=None, flowers_cls=['0', '1','2', '3','4'], 
                 batch_size=32, dim=(256,256), n_channels=3, shuffle=True):
        """
        Initialise the data generator
        """
        self.dim = dim
        self.batch_size = batch_size
        self.labels = {}
        self.list_IDs = []
        
        # glob through directory of each class 
        for i, cls in enumerate(flowers_cls):
            paths = glob.glob(os.path.join(data_path, cls, '*'))
            brk_point = int(len(paths)*0.8)
            if mode == 'train':
                paths = paths[:brk_point]
            else:
                paths = paths[brk_point:]
            if ablation is not None:
                paths = paths[:ablation]
            self.list_IDs += paths
            self.labels.update({p:i for p in paths})
            
        self.n_channels = n_channels
        self.n_classes = len(flowers_cls)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        delete_rows = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = io.imread(ID)
            img = img/255
            #if img.shape[0] > 256 and img.shape[1] > 256:
            img = cv2.resize(img, img_size)
            #else:
            #    delete_rows.append(i)
            #    continue
            X[i,] = img
          
            # Store class
            y[i] = self.labels[ID]
           
        
        #X = np.delete(X, delete_rows, axis=0)
        #y = np.delete(y, delete_rows, axis=0)
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# model architecture
model = Sequential()

model.add(Conv2D(64, (3,3), padding='same',input_shape=input_size))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3),padding='same'))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3),padding='same'))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3),padding='same'))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(1024, (3,3),padding='same'))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

In [ ]:
#compile model
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# summary
model.summary()

In [ ]:
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=5
)

In [ ]:
resnet.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# summary
resnet.summary()

In [ ]:
class roc_callback(Callback):
    
    def on_train_begin(self, logs={}):
        logs['val_auc'] = 0

    def on_epoch_end(self, epoch, logs={}):
        y_p = []
        y_v = []
        for i in range(len(validation_generator)):
            x_val, y_val = validation_generator[i]
            y_pred = self.model.predict(x_val)
            y_p.append(y_pred)
            y_v.append(y_val)
        y_p = np.concatenate(y_p)
        y_v = np.concatenate(y_v)
        roc_auc = roc_auc_score(y_v, y_p)
        print ('\nVal AUC for epoch{}: {}'.format(epoch, roc_auc))
        logs['val_auc'] = roc_auc

In [ ]:
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'


#EarlyStopping = EarlyStopping(monitor='val_loss', patience=21, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=2, save_best_only=True, save_weights_only=False, mode='max', period=1)

LR = ReduceLROnPlateau(monitor='categorical_accuracy', factor=0.2,patience=12, cooldown =5,verbose=2, min_lr=0.001)

auc_logger = roc_callback()

callbacks_list = [LR, auc_logger]

In [ ]:
#(self, mode='train', ablation=None, flowers_cls=['0', '1','2', '3','4'], 
  #               batch_size=32, dim=(256,256), n_channels=3, shuffle=True):
training_generator = DataGenerator('train', batch_size=128)
validation_generator = DataGenerator('val', batch_size=128)


In [ ]:
# train
model.fit(training_generator,
            batch_size=batch_size,
            epochs=150,
            callbacks=callbacks_list,
            validation_data=validation_generator,
            shuffle=True)